In [127]:
from textstat.textstat import textstat
import csv
import pandas
import matplotlib
#matplotlib.style.use('ggplot')
%matplotlib inline
import ast
pandas.options.display.max_colwidth = 100000

## Keep all topics

In [128]:
all_df = pandas.read_csv('data/all_candidates_nop.csv') 
#TOPICS = ['Immigration', 'Campaign Finance', 'Foreign Policy/National Security','Abortion']
#all_df = all_df[(all_df['top_topic'].isin(TOPICS))]
deduped_title = all_df.drop_duplicates('title') 

In [174]:
re_all = 'hillary|clinton|bernie|sanders|marco|rubio|donald|trump|ted|cruz|john|kasich'
clinton_only = deduped_title[(~deduped_title['title'].str.contains('bernie|sanders|marco|rubio|donald|trump|ted|cruz|john|kasich', case=False))]
trump_only = deduped_title[(~deduped_title['title'].str.contains('hillary|clinton|bernie|sanders|marco|rubio|ted|cruz|john|kasich', case=False))]
sanders_only = deduped_title[(~deduped_title['title'].str.contains('hillary|clinton|marco|rubio|donald|trump|ted|cruz|john|kasich', case=False))]
cruz_only = deduped_title[(~deduped_title['title'].str.contains('bernie|sanders|hillary|clinton|marco|rubio|donald|trump|john|kasich', case=False))]

In [130]:
print len(trump_only)
print len(clinton_only)
print len(sanders_only)
print len(cruz_only)

666
181
135
177


In [181]:
all_df = pandas.concat([clinton_only, trump_only, sanders_only, cruz_only])


## Remove AP stories that are longass updates

In [182]:
ap_updates = all_df[all_df['title'].str.contains("The Latest:")]['title'] 
print len(ap_updates)
all_df = all_df[~all_df['title'].str.contains("The Latest:")]
print len(all_df)
print len(all_df[all_df.org=='ap'])

34
1125
42


## only top 4 topics && only WSJ, NYT, Politico

In [183]:
CANDIDATES = ['sanders','cruz','clinton','trump']
ORGS = ['nyt', 'wsj', 'ap','cnn','fox']
TOPICS = ['Immigration', 'Campaign Finance', 'Foreign Policy/National Security','Abortion']
filtered_df = all_df[(all_df['top_topic'].isin(TOPICS))]
filtered_df = filtered_df[(filtered_df['org'].isin(ORGS))]
print len(filtered_df)

333


In [187]:
for c in CANDIDATES:
    print c, filtered_df[filtered_df.candidate == c]['flesch'].mean()


sanders 9.23703703704
cruz 8.49827586207
clinton 10.4214285714
trump 8.56116504854


## Get Top 10, Bottom 10, Middle 10


In [188]:
all_candidates = []
print "MID"
for c in CANDIDATES:
    print c.upper()
    sorted_c = filtered_df[filtered_df['candidate'] == c].sort_values('flesch')
    n = len(sorted_c)
    #print 'total', n
    #print 'mean', '%.2f' % sorted_c['flesch'].mean()
    low = sorted_c.head(int(10))
    high = sorted_c.tail(int(10))
    middle = sorted_c[((n/2)-5):(n/2+5)]
    print  
 
    print low[['org', 'top_topic', 'flesch']].to_string()
    #print low.org.value_counts().to_string()
    print
    #print low.top_topic.value_counts().to_string()
    print
    print "HIGH"
    print high[['org', 'top_topic', 'flesch']].to_string()
    #print high.org.value_counts().to_string()
    print
    print high.top_topic.value_counts().to_string()
    print
    print "MED"
    print middle[['org', 'top_topic', 'flesch']].to_string()
    #print middle.org.value_counts().to_string()
    #print
    #print middle.top_topic.value_counts().to_string()
    print
    print 
    low.to_csv('data/LowMiddleHigh/' + c + '_low.csv', \
                         columns=['title','url','candidate','org','top_topic','flesch','gunning_fog','body'])
    middle.to_csv('data/LowMiddleHigh/' + c + '_mid.csv', \
                         columns=['title','url','candidate','org','top_topic','flesch','gunning_fog','body'])
    high.to_csv('data/LowMiddleHigh/' + c + '_hi.csv', \
                         columns=['title','url','candidate','org','top_topic','flesch','gunning_fog','body'])
    all_candidates.append(low)
    all_candidates.append(middle)
    all_candidates.append(high)
    

all_candidates_df = pandas.concat(all_candidates)
all_candidates_df.to_csv('data/LowMiddleHigh/all.csv', \
                         columns=['title','url','candidate','org','top_topic','flesch','gunning_fog','body'])

#len(filtered_df[(filtered_df.candidate == 'cruz') & (filtered_df.flesch > 11)]\
#    .sort_values('flesch')[['org','top_topic','flesch']]

all_candidates_df[all_candidates_df.org =='ap'].to_csv('data/LowMiddleHigh/only_ap.csv',\
                                                       columns=['title','url','candidate','org','top_topic','flesch','gunning_fog','body'])

MID
SANDERS

      org                         top_topic  flesch
1380  wsj                  Campaign Finance     6.0
1582  wsj                  Campaign Finance     6.4
1349  nyt                  Campaign Finance     6.8
1480  nyt  Foreign Policy/National Security     6.8
1584  cnn                  Campaign Finance     7.9
1387  wsj                  Campaign Finance     8.0
1475  nyt                  Campaign Finance     8.0
1481  nyt                  Campaign Finance     8.0
1477  nyt                  Campaign Finance     8.7
1392  wsj                  Campaign Finance     8.8


HIGH
      org                         top_topic  flesch
1590  cnn                  Campaign Finance     9.9
1393  cnn                       Immigration    10.3
1482  nyt                       Immigration    10.7
1595  cnn                  Campaign Finance    10.7
1573  cnn  Foreign Policy/National Security    11.1
1579   ap                       Immigration    11.1
1357  wsj                  Campaign Finance 

## Get Top, Mid, Low as above but cut long stories


In [210]:
char_limited = filtered_df[filtered_df['body'].str.len() <= 10000]
print len(filtered_df)
print len(char_limited)
# IS EVERYTHING LONG CRAP?
char_long = filtered_df[filtered_df['body'].str.len() > 10000]
print len(char_long)
char_long.to_csv('data/LowMiddleHighCharLimit/longstories.csv', columns=['title','url','org','candidate','body'])

333
322
11


In [211]:
all_candidates = []  
for c in CANDIDATES:
    print c.upper()
    sorted_c = char_limited[char_limited['candidate'] == c].sort_values('flesch')
    n = len(sorted_c)
    print 'total', n
    print 'mean', '%.2f' % sorted_c['flesch'].mean()
    low = sorted_c.head(int(10))
    high = sorted_c.tail(int(10))
    middle = sorted_c[((n/2)-5):(n/2+5)]
    print 
    print "LOW"
    print low[['org', 'top_topic', 'flesch']].to_string()
    print
    print low.org.value_counts().to_string()
    print
    print low.top_topic.value_counts().to_string()
    print
    print "MED"
    print middle[['org', 'top_topic', 'flesch']].to_string()
    print
    print middle.org.value_counts().to_string()
    print
    print middle.top_topic.value_counts().to_string()
    print
    print 
    print "HIGH"
    print high[['org', 'top_topic', 'flesch']].to_string()
    print
    print high.org.value_counts().to_string()
    print
    print high.top_topic.value_counts().to_string()
    print
 
    low.to_csv('data/LowMiddleHighCharLimit/' + c + '_low.csv', \
                         columns=['title','url','candidate','org','top_topic','flesch','gunning_fog','body'])
    middle.to_csv('data/LowMiddleHighCharLimit/' + c + '_mid.csv', \
                         columns=['title','url','candidate','org','top_topic','flesch','gunning_fog','body'])
    high.to_csv('data/LowMiddleHighCharLimit/' + c + '_hi.csv', \
                         columns=['title','url','candidate','org','top_topic','flesch','gunning_fog','body'])
    all_candidates.append(low)
    all_candidates.append(middle)
    all_candidates.append(high)
    

all_candidates_df = pandas.concat(all_candidates)
all_candidates_df.to_csv('data/LowMiddleHighCharLimit/all.csv', \
                         columns=['title','url','candidate','org','top_topic','flesch','gunning_fog','body'])

#len(char_limited[(char_limited.candidate == 'cruz') & (char_limited.flesch > 11)]\
#    .sort_values('flesch')[['org','top_topic','flesch']]

all_candidates_df[all_candidates_df.org =='ap'].to_csv('data/LowMiddleHighCharLimit/only_ap.csv',\
                                                       columns=['title','url','candidate','org','top_topic','flesch','gunning_fog','body'])

SANDERS
total 27
mean 9.24

LOW
      org                         top_topic  flesch
1380  wsj                  Campaign Finance     6.0
1582  wsj                  Campaign Finance     6.4
1349  nyt                  Campaign Finance     6.8
1480  nyt  Foreign Policy/National Security     6.8
1584  cnn                  Campaign Finance     7.9
1387  wsj                  Campaign Finance     8.0
1475  nyt                  Campaign Finance     8.0
1481  nyt                  Campaign Finance     8.0
1477  nyt                  Campaign Finance     8.7
1392  wsj                  Campaign Finance     8.8

nyt    5
wsj    4
cnn    1

Campaign Finance                    9
Foreign Policy/National Security    1

MED
      org         top_topic  flesch
1477  nyt  Campaign Finance     8.7
1392  wsj  Campaign Finance     8.8
1476  nyt  Campaign Finance     8.8
1348  nyt  Campaign Finance     9.1
1356  nyt       Immigration     9.1
1377  nyt  Campaign Finance     9.1
1526  wsj       Immigration     9.